# SQLAlchemy ORM Declarative

### Here we "declare" our class tables.

We define a class for every table in the database.  

This of course is awesome for creating databases

In [2]:
# Dependencies
# ----------------------------------
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session
from sqlalchemy import func

In [3]:
# create declarative base
Base = declarative_base()

In [4]:
# tables yet?
Base.metadata.tables

immutabledict({})

In [5]:
# connect to database
db_path = "pets.db"
engine = create_engine(f"sqlite:///{db_path}")
conn = engine.connect()

In [6]:
# Now let's create our classes that mirror our desired tables
# ----------------------------------
class Dog(Base):
    __tablename__ = 'dog'
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    color = Column(String(255))
    age = Column(Integer)
    

class Cat(Base):
    __tablename__ = 'cat'
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    color = Column(String(255))
    age = Column(Integer)    

In [7]:
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
Base.metadata.create_all(engine)

# Use this to clear out the db
# ----------------------------------
# Base.metadata.drop_all(engine)

In [8]:
# current in memory tables
Base.metadata.tables

immutabledict({'dog': Table('dog', MetaData(bind=None), Column('id', Integer(), table=<dog>, primary_key=True, nullable=False), Column('name', String(length=255), table=<dog>), Column('color', String(length=255), table=<dog>), Column('age', Integer(), table=<dog>), schema=None), 'cat': Table('cat', MetaData(bind=None), Column('id', Integer(), table=<cat>, primary_key=True, nullable=False), Column('name', String(length=255), table=<cat>), Column('color', String(length=255), table=<cat>), Column('age', Integer(), table=<cat>), schema=None)})

In [9]:
# orm requires session so rollbacks can occur etc.
session = Session(bind=engine)

In [10]:
dir(session)

['_Session__binds',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_bind',
 '_after_attach',
 '_autoflush',
 '_before_attach',
 '_bulk_save_mappings',
 '_close_impl',
 '_conditional_expire',
 '_connection_for_bind',
 '_contains_state',
 '_delete_impl',
 '_deleted',
 '_dirty_states',
 '_enable_transaction_accounting',
 '_expire_state',
 '_expunge_states',
 '_flush',
 '_flush_warning',
 '_flushing',
 '_identity_cls',
 '_is_clean',
 '_merge',
 '_new',
 '_query_cls',
 '_register_altered',
 '_register_persistent',
 '_remove_newly_deleted',
 '_save_impl',
 '_save_or_update_impl',
 '_save_or_update_state',
 '_update_impl',
 '_validate_persi

### State of the current session

In [11]:
# any updates pending?
session.dirty

IdentitySet([])

In [12]:
# any inserts pending?
session.new

IdentitySet([])

## Create with Classes and Add

In [13]:
# create instances of the new classes
patterson = Dog(name='Patterson', color='white & red', age=11)
judson = Dog(name='Judson', color='white & red', age=10)
tobi = Cat(name='Tobi', color='orange', age=14)
oliver = Cat(name='Oliver', color='orange', age=12)

In [14]:
# add one item
session.add(patterson)

### Now let's see what the session looks like

In [15]:
print("new:\n", session.new, "\ndirty:\n", session.dirty)

new:
 IdentitySet([<__main__.Dog object at 0x000001B4B2A2FD08>]) 
dirty:
 IdentitySet([])


### Now let's bulk add

In [17]:
session.add_all([judson, tobi, oliver])

### Now let's see what the session looks like after all adds

In [18]:
print("new:\n", session.new, "\ndirty:\n", session.dirty)

new:
 IdentitySet([<__main__.Dog object at 0x000001B4B2A2FD08>, <__main__.Dog object at 0x000001B4B2A1E588>, <__main__.Cat object at 0x000001B4B2A29208>, <__main__.Cat object at 0x000001B4B2A1ED88>]) 
dirty:
 IdentitySet([])


### Commit and inspect

In [19]:
session.commit()
print("new:\n", session.new, "\ndirty:\n", session.dirty)

new:
 IdentitySet([]) 
dirty:
 IdentitySet([])


## Read with Query

In [20]:
help(session.query)

Help on method query in module sqlalchemy.orm.session:

query(*entities, **kwargs) method of sqlalchemy.orm.session.Session instance
    Return a new :class:`.Query` object corresponding to this
    :class:`.Session`.



In [21]:
# simple query of full table
cats = session.query(Cat)
for cat in cats:
    print(cat.name)

Tobi
Oliver


In [22]:
# return a tuple list like fetchall() with engine
session.query(Cat.name, Cat.age).all()

[('Tobi', 14), ('Oliver', 12)]

In [32]:
# most sql clauses have mathing function calls on session.query
dogs = session.query(Dog).order_by(Dog.name)
for dog in dogs:
    print(dog.name)

Judson
Patterson


In [40]:
# filter_by uses property operator value ( note single = [like sql does])
dogs = session.query(Dog).filter_by(name='Patterson')
for dog in dogs:
    print(dog.name)

Patterson


In [41]:
# filter uses Class.property notation and == [more Pythonic]
dogs = session.query(Dog).filter(Dog.name=='Patterson')
for dog in dogs:
    print(dog.name)

Patterson


In [43]:
# you can query just columns
for name, age in session.query(Dog.name, Dog.age):
    print(f"{name} is {age} years old")

Patterson is 11 years old
Judson is 10 years old


In [44]:
# getting counts
session.query(Dog).count()

2

In [47]:
# using functions i.e  select count(*) = func.count
session.query(func.count('*')).select_from(Dog).scalar()

2

In [59]:
# average age of all dogs
session.query(func.avg(Dog.age)).scalar()

10.5

## Update with Query & standard instance updating

updates and deletes first require a query to load the object in the session

In [51]:
oliver = session.query(Cat).filter_by(name='Oliver').first()
print(oliver, oliver.age)

<__main__.Cat object at 0x000002AA3A0B6FC8> 12


In [52]:
# current state
print("new:\n", session.new, "\ndirty:\n", session.dirty)

new:
 IdentitySet([]) 
dirty:
 IdentitySet([])


In [53]:
# update object
oliver.age = 13

In [54]:
# current state ( dirty changes once object updated)
print("new:\n", session.new, "\ndirty:\n", session.dirty)

new:
 IdentitySet([]) 
dirty:
 IdentitySet([<__main__.Cat object at 0x000002AA3A0B6FC8>])


In [60]:
session.commit()
# current state
print("new:\n", session.new, "\ndirty:\n", session.dirty)
engine.execute("select * from cat").fetchall()

new:
 IdentitySet([]) 
dirty:
 IdentitySet([])


[(1, 'Tobi', 'orange', 14), (2, 'Oliver', 'orange', 13)]

## Delete with Query and delete

In [72]:
engine.execute("select * from cat").fetchall()

[(1, 'Tobi', 'orange', 14), (2, 'Oliver', 'orange', 13)]

In [86]:
# don't post changes
# session.rollback()

In [87]:
cat = session.query(Cat).filter(Cat.id == 1).delete()

new:
 IdentitySet([]) 
dirty:
 IdentitySet([]) 
deleted:
 IdentitySet([])


In [89]:
session.commit()
engine.execute("select * from cat").fetchall()

[(2, 'Oliver', 'orange', 13)]

In [93]:
session.close()